In [1]:
import numpy as np
import pandas as pd
import scipy.stats as ss

In [2]:
import matplotlib.pyplot as plt
plt.rcParams['mathtext.fontset'] = 'cm'
plt.rcParams['font.sans-serif'] = 'Microsoft JhengHei'
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['axes.titlesize'] = 18
plt.rcParams['axes.labelsize'] = 16
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['legend.fontsize'] = 14

# 第二題

## 資料前處理

In [3]:
colname1 = names=['ID', 'Lon', 'Lat', 'EL', 'Name']
stn = pd.read_fwf('StnList.txt', names=colname1, encoding='Big5')
colname2 = ['Date']
colname2.extend(stn['Name'].to_list())
Temp = pd.read_fwf('19StnTmp.txt', names=colname2, index_col='Date')
Temp.index = pd.to_datetime(Temp.index.astype('str'), format='%Y%m%d')
Temp.head()

,鞍部,臺北,竹子湖,基隆,花蓮,蘇澳,宜蘭,臺南,高雄,嘉義,臺中,阿里山,大武,新竹,恆春,成功,日月潭,臺東,梧棲
Date,,,,,,,,,,,,,,,,,,,
1995-03-02,11.6,15.7,12.9,16.3,15.9,16.5,16.1,16.6,18.3,15.1,15.2,7.1,19.2,15.4,21.7,16.9,12.8,17.1,15.0
1995-03-24,17.6,21.3,18.9,20.4,24.4,22.3,22.4,24.3,24.6,23.6,23.6,12.6,25.6,21.0,25.1,25.1,19.3,25.1,22.5
1995-04-21,19.3,24.7,20.8,23.7,25.4,26.0,24.6,27.1,27.4,25.6,25.2,14.0,26.7,24.0,27.6,26.0,21.1,26.2,24.3
1995-04-22,21.6,26.5,22.7,25.0,26.1,26.5,26.1,27.9,27.7,26.5,27.1,16.1,27.0,25.9,27.6,26.6,22.3,26.6,27.4
1995-04-30,21.5,26.9,22.9,25.6,27.3,27.3,26.6,28.5,28.2,27.5,27.5,13.5,27.2,26.1,28.3,27.5,22.4,27.4,27.7


## 第A小題

In [4]:
# Temperature anomaly series of each station
stn0Mean = Temp.sub(Temp.mean())
# Mean temperature anomaly series of 19 stns without Feb. 29
Feb29idx = stn0Mean[(stn0Mean.index.month==2)&(stn0Mean.index.day==29)].index
tmpMean  = stn0Mean.mean(axis=1).drop(labels=Feb29idx)
JulDay = tmpMean.index.dayofyear.values

In [20]:
def coefficient(x, y):
    n = x.shape[1]
    mx = x.mean(axis=1, keepdims=True)
    my = y.mean()
    Matrix = np.cov(x) * n
    Vector = (x-mx) @ (y-my)
    coeff  = np.linalg.inv(Matrix) @ Vector
    beta0  = my - mx.flatten() @ coeff
    coeff  = np.concatenate((beta0, coeff), axis=None)
    return coeff

def SSErr(x, y, b):
    n = x.shape[0]
    y_hat = b[0]
    for i in range(1,n+1):
        y_hat = y_hat + b[i]*x[i-1]
    return np.sum((y_hat-y)**2)

In [24]:
x1 = np.array([np.sin(2*np.pi*JulDay/365),np.cos(2*np.pi*JulDay/365)])
y = tmpMean.values
beta = coefficient(x1, y)
SSErr(x1, y, beta)

4639.786668288079

In [25]:
x2 = np.array([np.sin(2*np.pi*JulDay/365),np.cos(2*np.pi*JulDay/365),
               np.sin(4*np.pi*JulDay/365),np.cos(4*np.pi*JulDay/365)])
beta = coefficient(x2, y)
SSErr(x2, y, beta)

4488.664775362229

In [26]:
y.shape

(2536,)

In [29]:
(4639.786668288079 - 4488.664775362229)*(2536-5)/(2*4488.664775362229)

42.60615685701998

In [32]:
ss.f.ppf(0.95, 2531, 2)

19.495330648145984